# Benchmark Results Viewer

This notebook displays benchmark results in a nicely formatted table.

## Usage
1. Specify JSON file path OR paste the JSON output from the benchmark (summary) log
2. Run the cells to see the formatted output

In [1]:
import json
from pathlib import Path
from typing import Any

## Specify file or paste your benchmark JSON here

In [38]:
# Load from file
# benchmark_file = Path("/Users/pokorny/JupyterLab/git/obi-output/circuit_extraction_benchmark_test4/initialize.circuit=N_10__top_nodes_dim6/neuron_set.sample_percentage=50.0__BENCHMARK__/benchmark_results.json")
# with benchmark_file.open("r") as f:
#     benchmark_data = json.load(f)

# OR paste the JSON from the benchmark summary log here
benchmark_json = '{"benchmarks": {"resolve_circuit": {"duration_s": 301.84, "mem_before_mb": 363.2, "mem_after_mb": 16270.74, "mem_delta_mb": 15907.54, "peak_mem_mb": 23077.01}, "add_node_set": {"duration_s": 0.82, "mem_before_mb": 16250.74, "mem_after_mb": 7805.61, "mem_delta_mb": -8445.13, "peak_mem_mb": 16250.74}, "split_subcircuit": {"duration_s": 53.45, "mem_before_mb": 7805.61, "mem_after_mb": 8185.47, "mem_delta_mb": 379.86, "peak_mem_mb": 9039.05}, "copy_morph_hoc_mod": {"duration_s": 17.83, "mem_before_mb": 8185.47, "mem_after_mb": 8185.73, "mem_delta_mb": 0.25, "peak_mem_mb": 8185.73}, "register_circuit_entity": {"duration_s": 0.1, "mem_before_mb": 8185.73, "mem_after_mb": 8185.73, "mem_delta_mb": 0.0, "peak_mem_mb": 8185.73}, "register_circuit_folder_asset": {"duration_s": 24.53, "mem_before_mb": 8185.73, "mem_after_mb": 8185.99, "mem_delta_mb": 0.26, "peak_mem_mb": 8185.99}, "run_circuit_folder_compression": {"duration_s": 402.41, "mem_before_mb": 8185.99, "mem_after_mb": 8185.99, "mem_delta_mb": 0.0, "peak_mem_mb": 8185.99}, "add_compressed_circuit_asset": {"duration_s": 6.51, "mem_before_mb": 8185.99, "mem_after_mb": 8185.99, "mem_delta_mb": 0.0, "peak_mem_mb": 8185.99}, "run_connectivity_matrix_extraction": {"duration_s": 0.1, "mem_before_mb": 8185.99, "mem_after_mb": 8185.99, "mem_delta_mb": 0.0, "peak_mem_mb": 8185.99}, "run_basic_connectivity_plots": {"duration_s": 0.1, "mem_before_mb": 8185.99, "mem_after_mb": 8185.99, "mem_delta_mb": 0.0, "peak_mem_mb": 8185.99}, "generate_overview_figures": {"duration_s": 0.1, "mem_before_mb": 8185.99, "mem_after_mb": 8185.99, "mem_delta_mb": 0.0, "peak_mem_mb": 8185.99}, "cleanup": {"duration_s": 0.1, "mem_before_mb": 8185.99, "mem_after_mb": 8185.99, "mem_delta_mb": 0.0, "peak_mem_mb": 8185.99}}, "benchmarked_time_s": 807.89, "total_execution_time_s": 807.99, "unbenchmarked_time_s": 0.1, "unbenchmarked_percentage": 0.0}'
benchmark_data = json.loads(benchmark_json)

## Display formatted results

In [39]:
def format_benchmark_results(benchmark_data: dict[str, Any]) -> None:
    """Display benchmark results in a nicely formatted table."""
    # Check if this is a summary (has 'benchmarks' key) or individual section
    if "benchmarks" in benchmark_data:
        # This is a summary
        benchmarks = benchmark_data["benchmarks"]
        benchmarked_time = benchmark_data.get("benchmarked_time_s")
        total_execution_time = benchmark_data.get("total_execution_time_s")
        unbenchmarked_time = benchmark_data.get("unbenchmarked_time_s")
        unbenchmarked_pct = benchmark_data.get("unbenchmarked_percentage")
        is_summary = True
    else:
        # This is an individual section benchmark
        benchmarks = benchmark_data
        benchmarked_time = sum(b["duration_s"] for b in benchmarks.values())
        total_execution_time = None
        unbenchmarked_time = None
        unbenchmarked_pct = None
        is_summary = False

    if not benchmarks:
        print("No benchmark data found.")
        return

    print("=" * 80)
    if is_summary:
        print("BENCHMARK SUMMARY")
    else:
        print("BENCHMARK SECTION")
    print("=" * 80)

    overall_peak_memory = 0
    for name, data in benchmarks.items():
        duration = data['duration_s']
        # Calculate percentage if total execution time is available
        if total_execution_time and total_execution_time > 0:
            pct = (duration / total_execution_time) * 100
            pct_str = f" ({pct:.1f}%)"
        else:
            pct_str = ""
        
        overall_peak_memory = max(overall_peak_memory, data['peak_mem_mb'])
        
        print(f"\n{name}:")
        print(f"  Duration:      {duration:>10.2f} s{pct_str}")
        print(f"  Memory Before: {data['mem_before_mb']:>10.2f} MB")
        print(f"  Memory After:  {data['mem_after_mb']:>10.2f} MB")
        print(f"  Memory Delta:  {data['mem_delta_mb']:>10.2f} MB")
        print(f"  Peak Memory:   {data['peak_mem_mb']:>10.2f} MB")

    if is_summary:
        print(f"\n{'─' * 80}")
        if benchmarked_time is not None:
            print(f"Benchmarked Time: {benchmarked_time:.2f} s")
        
        if total_execution_time is not None:
            print(f"\nUnbenchmarked Time: {unbenchmarked_time:.2f} s ({unbenchmarked_pct:.1f}%)")
            print(f"Total Execution Time: {total_execution_time:.2f} s")
        
        if overall_peak_memory > 0:
            print(f"Overall Peak Memory: {overall_peak_memory:.2f} MB")
    print("=" * 80)

In [40]:
# Display
try:
    format_benchmark_results(benchmark_data)
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
    print("Please check that you've pasted valid JSON.")

BENCHMARK SUMMARY

resolve_circuit:
  Duration:          301.84 s (37.4%)
  Memory Before:     363.20 MB
  Memory After:    16270.74 MB
  Memory Delta:    15907.54 MB
  Peak Memory:     23077.01 MB

add_node_set:
  Duration:            0.82 s (0.1%)
  Memory Before:   16250.74 MB
  Memory After:     7805.61 MB
  Memory Delta:    -8445.13 MB
  Peak Memory:     16250.74 MB

split_subcircuit:
  Duration:           53.45 s (6.6%)
  Memory Before:    7805.61 MB
  Memory After:     8185.47 MB
  Memory Delta:      379.86 MB
  Peak Memory:      9039.05 MB

copy_morph_hoc_mod:
  Duration:           17.83 s (2.2%)
  Memory Before:    8185.47 MB
  Memory After:     8185.73 MB
  Memory Delta:        0.25 MB
  Peak Memory:      8185.73 MB

register_circuit_entity:
  Duration:            0.10 s (0.0%)
  Memory Before:    8185.73 MB
  Memory After:     8185.73 MB
  Memory Delta:        0.00 MB
  Peak Memory:      8185.73 MB

register_circuit_folder_asset:
  Duration:           24.53 s (3.0%)
  Memory